In [70]:
import pandas as pd

listings = pd.read_csv('listings.csv')
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,7087327,https://www.airbnb.com/rooms/7087327,20151002231825,2015-10-03,Historic DC Condo-Walk to Capitol!,Professional pictures coming soon! Welcome to ...,NaN,Professional pictures coming soon! Welcome to ...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,18,NaN
1,975833,https://www.airbnb.com/rooms/975833,20151002231825,2015-10-03,Spacious Capitol Hill Townhouse,NaN,Beautifully renovated Capitol Hill townhouse. ...,Beautifully renovated Capitol Hill townhouse. ...,none,NaN,...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,strict,f,f,1,2.11
2,8249488,https://www.airbnb.com/rooms/8249488,20151002231825,2015-10-03,Spacious/private room for single,This is an ideal room for a single traveler th...,NaN,This is an ideal room for a single traveler th...,none,NaN,...,NaN,f,NaN,NaN,f,flexible,f,f,1,1.00
3,8409022,https://www.airbnb.com/rooms/8409022,20151002231825,2015-10-03,A wonderful bedroom with library,Prime location right on the Potomac River in W...,NaN,Prime location right on the Potomac River in W...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,1,NaN
4,8411173,https://www.airbnb.com/rooms/8411173,20151002231825,2015-10-03,Downtown Silver Spring,"Hi travellers! I live in this peaceful spot, b...",This is a 750 sq ft 1 bedroom 1 bathroom. Whi...,"Hi travellers! I live in this peaceful spot, b...",none,Silver Spring is booming. You can walk to a n...,...,NaN,f,NaN,NaN,f,flexible,f,f,1,NaN


In [71]:
dc_listings = listings.loc[:,['host_response_rate','host_acceptance_rate',
                              'host_listings_count','accommodates','room_type',
                             'bedrooms','bathrooms','beds','price','cleaning_fee',
                              'security_deposit','minimum_nights','maximum_nights',
                             'number_of_reviews','latitude','longitude','city','zipcode','state']]
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
0,92%,91%,26,4,Entire home/apt,1.0,1.0,2.0,$160.00,$115.00,$100.00,1,1125,0,38.890046,-77.002808,Washington,20003,DC
1,90%,100%,1,6,Entire home/apt,3.0,3.0,3.0,$350.00,$100.00,NaN,2,30,65,38.880413,-76.990485,Washington,20003,DC
2,90%,100%,2,1,Private room,1.0,2.0,1.0,$50.00,NaN,NaN,2,1125,1,38.955291,-76.986006,Hyattsville,20782,MD
3,100%,NaN,1,2,Private room,1.0,1.0,1.0,$95.00,NaN,NaN,1,1125,0,38.872134,-77.019639,Washington,20024,DC
4,92%,67%,1,4,Entire home/apt,1.0,1.0,1.0,$50.00,$15.00,$450.00,7,1125,0,38.996382,-77.041541,Silver Spring,20910,MD


In [72]:
#The K-nearest neighbors algorithm
"""
            Euclidian Distance
d = \sqrt{(q_1-p_1)^2 + (q_2-p_2)^2 + \cdots + (q_n-p_n)^2}
q_1 to q_n represent the feature values for one observation
p_1 to p_n represent the feature values for the other observation


Let's look at just one column from our data set
d = \sqrt{(q_1 – p_1)^2}
d = | q_1 – p_1 |
we want to find the absolute value of the difference between the observation 
and the data point we want to predict for the feature we’re using


let’s say the living space that we want to rent can accommodate three people
we’ll first calculate the distance, using just the accommodates feature, 
between the first living space in the data set and our own.
"""

import numpy as np
our_acc_value = 3
first_living_space_value = dc_listings.loc[0,'accommodates']
first_living_space_value

4

In [73]:
first_distance = np.abs(first_living_space_value - our_acc_value)
first_distance

1

In [74]:
"""
The smallest possible Euclidian distance is zero, which would mean the observation we are comparing to is identical to ours, 
so the result we’ve gotten here makes sense. 
In isolation, however, the value doesn’t mean much unless we know how it compares to other values. 
Let’s calculate the Euclidean distance for each observation in our data set
"""
dc_listings['distance'] = np.abs(dc_listings.accommodates - our_acc_value)
dc_listings.distance.value_counts().sort_index()

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [75]:
"""
There are 461 listings that have a distance of 0, and thus accommodate the same number of people as our listing. 
Those listings might be a good place to start.
If we just used the first five values with a distance of 0, our predictions would be biased to the existing ordering of the data set. 
Instead, we’ll randomize the ordering of the observations and then select the first five rows with a distance of 0

We’ll also use the random_state parameter which just gives us a reproducible random order 
so anyone can follow along and get the exact same results.
"""
dc_listings = dc_listings.sample(frac=1,random_state=0)
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance
2645,100%,100%,1,3,Entire home/apt,1.0,1.0,1.0,$75.00,$50.00,$300.00,7,180,24,38.927614,-76.995799,Washington,20017,DC,0
1503,100%,94%,2,7,Entire home/apt,2.0,2.0,3.0,$100.00,NaN,$100.00,1,1125,1,38.878154,-77.020250,Washington,20024,DC,4
1791,100%,86%,1,3,Private room,1.0,1.0,1.0,$95.00,$40.00,$100.00,2,1125,16,38.958706,-77.084936,Washington,20015,DC,0
3539,100%,100%,1,5,Entire home/apt,3.0,2.5,3.0,$265.00,NaN,NaN,2,7,0,38.885398,-77.004258,Washington,20003,DC,2
489,100%,64%,1,2,Entire home/apt,1.0,1.0,1.0,$111.00,$59.00,$150.00,1,1125,6,38.911885,-77.018052,Washington,20001,DC,1


In [76]:
dc_listings = dc_listings.sort_values('distance')
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance
2645,100%,100%,1,3,Entire home/apt,1.0,1.0,1.0,$75.00,$50.00,$300.00,7,180,24,38.927614,-76.995799,Washington,20017,DC,0
2825,100%,100%,1,3,Entire home/apt,3.0,2.0,2.0,$120.00,NaN,NaN,1,1125,0,38.929772,-77.026908,Washington,20010,DC,0
2145,100%,100%,1,3,Private room,1.0,2.0,2.0,$90.00,NaN,NaN,1,1125,55,38.900895,-77.001037,Washington,20002,DC,0
2541,NaN,NaN,1,3,Private room,1.0,1.0,1.0,$50.00,$25.00,NaN,1,1125,1,38.939758,-77.014023,Washington,20011,DC,0
3349,90%,100%,1,3,Entire home/apt,1.0,1.0,1.0,$105.00,$50.00,$100.00,1,1125,7,38.933036,-77.073194,Washington,20016,DC,0


In [77]:
dc_listings.price.head()

2645     $75.00
2825    $120.00
2145     $90.00
2541     $50.00
3349    $105.00
Name: price, dtype: object

In [78]:
"""
Before we can take the average of our prices, you’ll notice that our price column has the object type, 
due to the fact that the prices have dollar signs and commas
"""
dc_listings['price'] = dc_listings.price.str.replace("\$|,",'').astype(float)
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance
2645,100%,100%,1,3,Entire home/apt,1.0,1.0,1.0,75.0,$50.00,$300.00,7,180,24,38.927614,-76.995799,Washington,20017,DC,0
2825,100%,100%,1,3,Entire home/apt,3.0,2.0,2.0,120.0,NaN,NaN,1,1125,0,38.929772,-77.026908,Washington,20010,DC,0
2145,100%,100%,1,3,Private room,1.0,2.0,2.0,90.0,NaN,NaN,1,1125,55,38.900895,-77.001037,Washington,20002,DC,0
2541,NaN,NaN,1,3,Private room,1.0,1.0,1.0,50.0,$25.00,NaN,1,1125,1,38.939758,-77.014023,Washington,20011,DC,0
3349,90%,100%,1,3,Entire home/apt,1.0,1.0,1.0,105.0,$50.00,$100.00,1,1125,7,38.933036,-77.073194,Washington,20016,DC,0


In [79]:
mean_price = dc_listings.price.iloc[:5].mean()
mean_price

88.0

In [120]:
#Evaluating Our Model
"""
Split the data set into 2 partitions:
    A training set: contains the majority of the rows (75%)
    A test set: contains the remaining minority of the rows (25%)

Let's start removing the distance column we added earlier, and spliting the data.
No need to worry about randomizing the rows, since they’re still randomized from earlier
"""
dc_listings.drop('distance', axis=1)
train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]

def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dc_listings[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return predicted_price

In [121]:
test_df['predicted_price'] = test_df.accommodates.apply(predict_price, feature_column='accommodates')

In [122]:
#predict_price(5,'accommodates')
test_df

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,...,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance,predicted_price
2850,100%,80%,1,1,Shared room,1.0,1.0,1.0,40.0,NaN,...,2,1125,105,38.930875,-77.027756,Washington,20010,DC,2,83.6
2279,100%,0%,2,1,Shared room,1.0,1.0,1.0,45.0,NaN,...,1,1125,8,38.919616,-77.039691,Washington,20009,DC,2,83.6
2771,100%,81%,9,5,Entire home/apt,2.0,2.0,3.0,217.0,$200.00,...,3,730,19,38.925325,-77.034632,Washington,20009,DC,2,340.4
910,99%,88%,16,5,Entire home/apt,2.0,2.0,3.0,415.0,NaN,...,1,1125,1,38.904464,-77.058633,Washington,20007,DC,2,340.4
2434,92%,94%,30,5,Entire home/apt,1.0,1.0,3.0,275.0,$150.00,...,3,1125,9,38.913012,-77.048436,Washington,20008,DC,2,340.4
965,100%,94%,7,5,Entire home/apt,1.0,1.0,2.0,145.0,$55.00,...,4,1000,74,38.913958,-77.045686,Washington,20008,DC,2,340.4
1305,100%,100%,1,1,Private room,1.0,1.0,1.0,100.0,$20.00,...,2,1125,21,38.918407,-77.029875,Washington,20009,DC,2,83.6
2513,100%,100%,4,1,Private room,1.0,1.5,1.0,80.0,$10.00,...,2,1125,8,38.906123,-77.013281,Washington,20001,DC,2,83.6
2118,53%,92%,1,5,Entire home/apt,1.0,1.0,1.0,115.0,$50.00,...,1,3,43,38.893295,-76.997139,Washington,20002,DC,2,340.4
345,92%,76%,206,5,Entire home/apt,2.0,2.0,3.0,324.0,$100.00,...,3,1125,2,38.900239,-77.050573,Washington,20037,DC,2,340.4


In [126]:
#Using RMSE to Evaluate Our Model
"""
For many prediction tasks, we want to penalize predicted values that are further away from 
the actual value much more than those that are closer to the actual value.

RMSE = \sqrt {\dfrac{ (actual_1-predicted_1)^2 + (actual_2-predicted_2)^2 + \cdots + (actual_n-predicted_n)^2 }{ n }}
"""
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
test_df.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,...,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance,predicted_price,squared_error
2850,100%,80%,1,1,Shared room,1.0,1.0,1.0,40.0,NaN,...,1125,105,38.930875,-77.027756,Washington,20010,DC,2,83.6,1900.96
2279,100%,0%,2,1,Shared room,1.0,1.0,1.0,45.0,NaN,...,1125,8,38.919616,-77.039691,Washington,20009,DC,2,83.6,1489.96
2771,100%,81%,9,5,Entire home/apt,2.0,2.0,3.0,217.0,$200.00,...,730,19,38.925325,-77.034632,Washington,20009,DC,2,340.4,15227.56
910,99%,88%,16,5,Entire home/apt,2.0,2.0,3.0,415.0,NaN,...,1125,1,38.904464,-77.058633,Washington,20007,DC,2,340.4,5565.16
2434,92%,94%,30,5,Entire home/apt,1.0,1.0,3.0,275.0,$150.00,...,1125,9,38.913012,-77.048436,Washington,20008,DC,2,340.4,4277.16


In [132]:
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

212.9892796705153

In [133]:
#Comparing different models

for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print("RMSE for the {} column: {}".format(feature,rmse))

RMSE for the accommodates column: 212.9892796705153
RMSE for the bedrooms column: 216.49048609414766
RMSE for the bathrooms column: 216.89419042215704
RMSE for the number_of_reviews column: 240.2152831433485


In [144]:
"""
So far, we’ve been training our model with only one feature, 
which is known as a univariate model. 
For more accuracy, we can have it assess multiple features at the same time, 
which is known as a multivariate model
"""
normalized_listings  = pd.read_csv('listings.csv')
normalized_listings  = normalized_listings .loc[:,['accommodates','bedrooms',
                              'bathrooms','beds','price',
                             'minimum_nights','maximum_nights','number_of_reviews',]]

normalized_listings ['price'] = normalized_listings .price.str.replace("\$|,",'').astype(float)
normalized_listings .info()
normalized_listings .head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723 entries, 0 to 3722
Data columns (total 8 columns):
accommodates         3723 non-null int64
bedrooms             3702 non-null float64
bathrooms            3696 non-null float64
beds                 3712 non-null float64
price                3723 non-null float64
minimum_nights       3723 non-null int64
maximum_nights       3723 non-null int64
number_of_reviews    3723 non-null int64
dtypes: float64(4), int64(4)
memory usage: 232.7 KB


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,160.0,1,1125,0
1,6,3.0,3.0,3.0,350.0,2,30,65
2,1,1.0,2.0,1.0,50.0,2,1125,1
3,2,1.0,1.0,1.0,95.0,1,1125,0
4,4,1.0,1.0,1.0,50.0,7,1125,0


In [145]:
normalized_listings = normalized_listings.sample(frac=1,random_state=0)
norm_train_df = normalized_listings.copy().iloc[0:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]

In [157]:
#Calculating Euclidean distance with multiple features
from scipy.spatial import distance
first_listing = normalized_listings.iloc[0][['accommodates', 'bathrooms']]
fifth_listing = normalized_listings.iloc[50][['accommodates', 'bathrooms']]
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)
print(first_listing,"\n",fifth_listing,"\n",first_fifth_distance)


accommodates    3.0
bathrooms       1.0
Name: 2645, dtype: float64 
 accommodates    2.0
bathrooms       1.0
Name: 1420, dtype: float64 
 1.0


In [167]:
#Creating a multivariate KNN model
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    cols = ['accommodates', 'bathrooms']
    return predicted_price.cols

cols = ['accommodates', 'bathrooms']
norm_test_df['predicted_price'] = norm_test_df['accommodates', 'bathrooms'].apply(predict_price_multivariate,feature_columns=cols,axis=1)
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

KeyError: ('accommodates', 'bathrooms')

In [168]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(algorithm='brute')
knn.fit(train_features, train_target)


NameError: name 'train_features' is not defined